In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
def initialize_dataset(addr = "https://www.the-numbers.com/movie/budgets/all") :
    page = requests.get(addr)
    soup = BeautifulSoup(page.content, 'lxml')
    table = soup.find("table")
    cols = []
    for item in table.find_all("th"):
        cols.append(item.text)
        cols[0] = 'Rank'
        
    df = pd.DataFrame(columns= cols)
    return (df, cols)

def get_table_values(df, cols, addr = "https://www.the-numbers.com/movie/budgets/all") :
    page = requests.get(addr)
    soup = BeautifulSoup(page.content, 'lxml')
    table = soup.find("table")
    
    for row in table.find_all('tr'):
        values = [[data.text for data in row.find_all('td')]]
        try:
            df = df.append(pd.DataFrame(values, columns= cols))
        except : 
            continue
    return df

df, cols = initialize_dataset()
dataset = get_table_values(df, cols)

visited = ["https://www.the-numbers.com/movie/budgets/all"]

flag = True
while flag :
    url = visited [-1]
    page = requests.get(url)

    soup = BeautifulSoup(page.content, 'lxml')

    pagination = soup.find("div", {"class" : "pagination"})
    for next in pagination.find_all("a", {"rel" : "nofollow"}) :
        next_page = "https://www.the-numbers.com" + next['href']
        if next_page not in visited:
            dataset = get_table_values(dataset, cols, next_page)
            visited.append(next_page)

    if url == 'https://www.the-numbers.com/movie/budgets/all/6101' :
        flag = False

dataset.to_csv('datasets/budget_income.csv', index= False)